# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [1]:
%run ../config/bootstrap.py

In [2]:
import pandas as pd
from datetime import datetime 
from utils import get_project_root, save_parquet_in_chunks  
import basedosdados as bd

In [4]:
project_root = get_project_root() 
billing_id = 'trabalho-pratico-ml'

Para fazer login:
``` shell
gcloud auth application-default login
```

In [6]:
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100

# %sql bigquery://genuine-orb-470122-v0
%sql bigquery://trabalho-pratico-ml

### Testing

In [7]:
%%sql  
SELECT ano, count(*) as qt FROM `basedosdados.br_ms_sinan.microdados_dengue` group by 1 order by 1 

Running query in 'bigquery://trabalho-pratico-ml'

E0000 00:00:1763327500.880652   87582 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


26 rows affected.

,ano,qt
0,2000,172855
1,2001,488590
2,2002,897093
3,2003,416609
4,2004,136867
5,2005,261501
6,2006,411022
7,2007,717097
8,2008,919324
9,2009,600658


# bi_query

### all classes (mainly low_risk)

In [8]:
%%sql df_22 <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where classificacao_final in ('10', '11', '12') and evolucao_caso is not null
and ano = 2022

Running query in 'bigquery://trabalho-pratico-ml'

1192119 rows affected.

In [9]:
df_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1192119 entries, 0 to 1192118
Data columns (total 45 columns):
 #   Column                         Non-Null Count    Dtype 
---  ------                         --------------    ----- 
 0   sigla_uf_residencia            1192119 non-null  object
 1   idade_paciente                 1192119 non-null  object
 2   sexo_paciente                  1192113 non-null  object
 3   raca_cor_paciente              1192110 non-null  object
 4   gestante_paciente              1191823 non-null  object
 5   possui_doenca_autoimune        1192067 non-null  object
 6   possui_diabetes                1192067 non-null  object
 7   possui_doencas_hematologicas   1192067 non-null  object
 8   possui_hepatopatias            1192067 non-null  object
 9   possui_doenca_renal            1192067 non-null  object
 10  possui_hipertensao             1192067 non-null  object
 11  possui_doenca_acido_peptica    1192067 non-null  object
 12  apresenta_febre             

In [ ]:
save_parquet_in_chunks(df_22, "data/2_silver/2022_all_classes", 50, compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1192119 [00:00<?, ?linhas/s]

✔️ 2022_all_classes_part_1.parquet salvo com 3.96 MB (1192119 linhas)
✅ Todos os arquivos salvos com sucesso.


In [11]:
del(df_22)

## alarm

In [12]:
%%sql df_alarm <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where classificacao_final = '11' and evolucao_caso != '2'
and evolucao_caso is not null
and ano in (2020, 2021, 2023, 2024, 2025) -- Sem 2022 para não repetir dados

Running query in 'bigquery://trabalho-pratico-ml'

186298 rows affected.

In [13]:
save_parquet_in_chunks(df_alarm, "data/2_silver/2020-2025_alarm", 50, compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/186298 [00:00<?, ?linhas/s]

✔️ 2020-2025_alarm_part_1.parquet salvo com 0.66 MB (186298 linhas)
✅ Todos os arquivos salvos com sucesso.


In [14]:
del(df_alarm)

## severe

In [15]:
%%sql df_severe <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where (classificacao_final = '12' or (evolucao_caso = '2' and classificacao_final in ('10', '11'))) -- Casos de Dengue Grave ou Evolução para óbito
and ano in (2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
            2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
            2020, 2021, 2023, 2024, 2025) -- Sem 2022 para não repetir dados

Running query in 'bigquery://trabalho-pratico-ml'

28324 rows affected.

In [16]:
save_parquet_in_chunks(df_severe, "data/2_silver/2000-2025_severe", 50, compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/28324 [00:00<?, ?linhas/s]

✔️ 2000-2025_severe_part_1.parquet salvo com 0.15 MB (28324 linhas)
✅ Todos os arquivos salvos com sucesso.


In [17]:
del(df_severe)